In [1]:
import pandas as pd
import os

from dotenv import load_dotenv
from azure.storage.blob import BlobServiceClient
from azure.storage.blob import ContainerClient

In [2]:
load_dotenv()

account_url = os.getenv('AZURE_ACCOUNT_URL', 'https://your-account.blob.core.windows.net')
account_name = os.getenv('AZURE_ACCOUNT_NAME', 'your-account-name')
container = os.getenv('AZURE_CONTAINER', 'your-container-name')
sas_token = os.getenv('AZURE_SAS_TOKEN', 'your-sas-token-here')

In [3]:
# Connect to Azure Blob Storage
blob_service_client = BlobServiceClient(account_url, credential=sas_token)
container_client = blob_service_client.get_container_client(container)

In [4]:
def list_all_videos_in_azure(container_client):
    """
    Lists all blobs in the given Azure Blob Storage container.
    Returns a DataFrame with blob name and size.
    """
    print("📦 Listing all blobs in container...")

    blobs = []
    for blob in container_client.list_blobs():
        blobs.append({
            "blob_name": blob.name,
            "size_bytes": blob.size,
            "last_modified": blob.last_modified
        })

    df_blobs = pd.DataFrame(blobs)
    print(f"✅ Found {len(df_blobs)} blobs in total.")
    print(df_blobs.head())

    return df_blobs

In [5]:
container_url = f"https://{account_name}.blob.core.windows.net/{container}?{sas_token}"

container_client = ContainerClient.from_container_url(container_url)

df_all = list_all_videos_in_azure(container_client)

📦 Listing all blobs in container...
✅ Found 162508 blobs in total.
                              blob_name  size_bytes             last_modified
0  banorte_cam1-2024-10-24 16:49:52.mp4     5094070 2024-11-20 20:19:11+00:00
1  banorte_cam1-2024-10-24 16:52:25.mp4     1690592 2024-11-20 20:22:27+00:00
2  banorte_cam1-2024-10-24 16:52:52.mp4     3817924 2024-11-20 20:11:44+00:00
3  banorte_cam1-2024-10-24 16:54:37.mp4     3319540 2024-11-20 20:06:34+00:00
4  banorte_cam1-2024-10-24 16:56:22.mp4     5969510 2024-11-20 20:03:51+00:00


In [6]:
df_all.to_csv('./../../data/azure_names.csv', index=False)